# Prediction on bigger flight (Raster) with New Full Model

In [1]:
from fastbook import *
import fiona
import rasterio as rio  
import rasterio.mask
from osgeo import gdal


c:\Users\oirys\miniforge3\envs\DISCOV_prediction\lib\site-packages\fastbook\__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
c:\Users\oirys\miniforge3\envs\DISCOV_prediction\lib\site-packages\fastbook\__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


# Set input and Output filename 

In [2]:
filename = "Gafanha_Low_LowRes"
output_name = "Gaf_Low_BiCOME"

learn = load_learner('../models/DISCOV_BiCOME.pkl')
categories = learn.dls.vocab
categories

['Bacillariophyceae', 'Chlorophyta', 'Magnoliopsida', 'Phaeophyta', 'Rhodophyta', 'Sediment', 'SunGlint', 'Water']

In [3]:
with fiona.open('../Data/shp/mask/' + filename + '.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rio.open('../Data/img/' + filename + '.tif') as ds:
    arr, out_transform = rasterio.mask.mask(ds, shapes, crop=True)
    out_meta = ds.meta

gt = [out_transform[2],out_transform[0],out_transform[1],
 out_transform[5],out_transform[3],out_transform[4]]


lowtif = gdal.Open('../Data/img/' + filename + '.tif')
proj = lowtif.GetProjection()
del lowtif 
gt

[-8.743678352905512,
 3.9002362204679466e-07,
 0.0,
 40.59793313496078,
 0.0,
 -2.9730920060172317e-07]

In [4]:

Arr_Max=np.max(arr,axis=0)
Arr_Min=np.min(arr,axis=0)

arr_std=(arr - Arr_Min)/(Arr_Max - Arr_Min)


C:\Users\oirys\AppData\Local\Temp\ipykernel_53740\3415335001.py:4: RuntimeWarning: invalid value encountered in divide
  arr_std=(arr - Arr_Min)/(Arr_Max - Arr_Min)


In [5]:
df_NAN = pd.DataFrame()
df_NAN['Reflectance_444'] = arr[0].ravel()
df_NAN['Reflectance_475'] = arr[1].ravel() 
df_NAN['Reflectance_531'] = arr[2].ravel() 
df_NAN['Reflectance_560'] = arr[3].ravel() 
df_NAN['Reflectance_650'] = arr[4].ravel() 
df_NAN['Reflectance_668'] = arr[5].ravel() 
df_NAN['Reflectance_705'] = arr[6].ravel() 
df_NAN['Reflectance_717'] = arr[7].ravel() 
df_NAN['Reflectance_740'] = arr[8].ravel() 
df_NAN['Reflectance_842'] = arr[9].ravel()

In [6]:
df_NAN.replace([0,65535], np.nan, inplace=True)

df_NAN['Reflectance_Stan_444'] = arr_std[0].ravel()
df_NAN['Reflectance_Stan_475'] = arr_std[1].ravel() 
df_NAN['Reflectance_Stan_531'] = arr_std[2].ravel() 
df_NAN['Reflectance_Stan_560'] = arr_std[3].ravel() 
df_NAN['Reflectance_Stan_650'] = arr_std[4].ravel() 
df_NAN['Reflectance_Stan_668'] = arr_std[5].ravel() 
df_NAN['Reflectance_Stan_705'] = arr_std[6].ravel() 
df_NAN['Reflectance_Stan_717'] = arr_std[7].ravel() 
df_NAN['Reflectance_Stan_740'] = arr_std[8].ravel() 
df_NAN['Reflectance_Stan_842'] = arr_std[9].ravel()

In [7]:
df_NAN['NDVI'] = (df_NAN['Reflectance_842']-df_NAN['Reflectance_650'])/(df_NAN['Reflectance_842']+df_NAN['Reflectance_650'])
df_NAN['NDVI_Stan'] = (df_NAN['Reflectance_Stan_842']-df_NAN['Reflectance_Stan_650'])/(df_NAN['Reflectance_Stan_842']+df_NAN['Reflectance_Stan_650'])

In [8]:
df = df_NAN.dropna()

In [9]:
df_test_nan_nrum = df_NAN
df_test_nan_nrum['ID'] = np.arange(len(df_test_nan_nrum))
df_test_nrum = df_test_nan_nrum.dropna()

In [10]:
df.shape

(2463151, 22)

In [11]:
ID_l=list(df_test_nrum['ID'])

In [12]:
dl = learn.dls.test_dl(df, bs=4000)
preds,_ = learn.get_preds(dl=dl)

In [13]:
class_idxs = preds.argmax(axis=1)
# res = [learn.dls.vocab[c] for c in class_idxs]

In [14]:
class_probs= preds.max(axis=1)

In [15]:
class_probs=class_probs.values

# Prediction Plot 

In [16]:
type(class_idxs.tolist())

list

In [17]:
NumPred= class_idxs.tolist()
PredProbs =class_probs.tolist()

In [18]:
res_df= pd.DataFrame(list(zip(NumPred, ID_l,PredProbs)),columns =['Pred_ID','ID','Prob'])

In [19]:
res_df

,Pred_ID,ID,Prob
0,2,10063,0.996501
1,2,10064,0.993642
2,2,11827,0.999338
3,2,11828,0.998681
4,2,11829,0.996831
...,...,...,...
2463146,2,4625820,0.998297
2463147,2,4625821,0.997265
2463148,2,4627579,0.995977
2463149,2,4627580,0.998293


In [20]:
df_NAN['ID']= np.arange(len(df_NAN))

In [21]:
res_input_df = pd.merge(df_NAN,res_df, how='left', on = 'ID')

In [22]:
res_input_df

,Reflectance_444,Reflectance_475,Reflectance_531,Reflectance_560,Reflectance_650,Reflectance_668,Reflectance_705,Reflectance_717,Reflectance_740,Reflectance_842,Reflectance_Stan_444,Reflectance_Stan_475,Reflectance_Stan_531,Reflectance_Stan_560,Reflectance_Stan_650,Reflectance_Stan_668,Reflectance_Stan_705,Reflectance_Stan_717,Reflectance_Stan_740,Reflectance_Stan_842,NDVI,NDVI_Stan,ID,Pred_ID,Prob
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4634885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4634885,NaN,NaN
4634886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4634886,NaN,NaN
4634887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4634887,NaN,NaN
4634888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4634888,NaN,NaN


In [23]:
Pred_arr = np.asarray(res_input_df['Pred_ID'])

In [24]:
Pred_arr=Pred_arr+1

In [25]:
arr.shape

(10, 2626, 1765)

In [26]:
Prob_arr = np.asarray(res_input_df['Prob'])
Prob_ras = Prob_arr.reshape(arr.shape[1], arr.shape[2])

In [27]:
Pred_ras = Pred_arr.reshape(arr.shape[1], arr.shape[2])

# Prediction Save

In [28]:
# export
def ensure_directory_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory {directory_path} created.")
    else:
        print(f"Directory {directory_path} already exists.")

ensure_directory_exists("../Output/Pred") 

driver = gdal.GetDriverByName("GTiff")
driver.Register()
outds = driver.Create("../Output/Pred/" + output_name +"_prob.tif", xsize = Prob_ras.shape[1],
                      ysize = Prob_ras.shape[0], bands = 1, 
                      eType = gdal.GDT_Float32)

outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(Prob_ras)
outband.SetNoDataValue(65535)
outband.FlushCache()

# close your datasets and bands!!!
outband = None
outds = None

driver = gdal.GetDriverByName("GTiff")
driver.Register()
outds = driver.Create("../Output/Pred/" + output_name +"_pred.tif", xsize = Pred_ras.shape[1],
                      ysize = Pred_ras.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(Pred_ras)
outband.SetNoDataValue(65535)
outband.SetNoDataValue(32767)
outband.FlushCache()
# close your datasets and bands!!!
outband = None
outds = None

Directory ../Output/Pred already exists.
